In [1]:
import pandas as pd

In [2]:
ny_map = pd.read_json("ny_map_data_res80.json")

In [3]:
len(ny_map)

6400

In [4]:
ny_map.head()

,county_name,lat,lng,state
0,Pike,41.403671,-74.984937,PA
1,Kawartha Lakes,44.788228,-79.055823,ON
2,Montgomery,42.920886,-74.476076,NY
3,Chautauqua,42.162278,-79.666456,NY
4,Rensselaer,42.862532,-73.661899,NY


In [15]:
ny_map["lng_rank"] = ny_map.lng.rank(method = "dense")
ny_map["lat_rank"] = ny_map.lat.rank(method = "dense")

In [19]:
ny_map_restricted = ny_map[ny_map.state == "NY"]

In [24]:
ny_map_restricted.groupby("county_name").count().sort_values("lat").head()

,lat,lng,state,lng_rank,lat_rank
county_name,,,,,
New York,1,1,1,1,1
Bronx,2,2,2,2,2
Richmond,3,3,3,3,3
Kings,4,4,4,4,4
Queens,5,5,5,5,5


In [25]:
ny_map[ny_map.county_name == "Kings"]

,county_name,lat,lng,state,lng_rank,lat_rank
2491,Kings,40.645063,-73.967215,NY,59.0,4.0
2959,Kings,40.645063,-73.865443,NY,60.0,4.0
5992,Kings,40.703418,-73.967215,NY,59.0,5.0
6332,Kings,40.586709,-73.967215,NY,59.0,3.0


In [26]:
ny_map[ny_map.county_name == "Queens"]

,county_name,lat,lng,state,lng_rank,lat_rank
157,Queens,40.703418,-73.865443,NY,60.0,5.0
1205,Queens,40.761772,-73.865443,NY,60.0,6.0
3393,Queens,40.645063,-73.763671,NY,61.0,4.0
3554,Queens,40.703418,-73.763671,NY,61.0,5.0
4335,Queens,40.761772,-73.763671,NY,61.0,6.0
